In [2]:
import os
import sys
sys.path.insert(0,"..")
 
import json
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import matplotlib.ticker as ticker
import torch
from matplotlib.ticker import MaxNLocator
import torch.nn.functional as F

import config
from util import set_seed, build_dataset, build_model, split_logits_labels

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

set_seed(66)

66

In [1]:
dataset_name = "cifar10"
model_name = "resnet18"

calibloader, testloader = build_dataset(dataset_name)
model = build_model(model_name, dataset_name, device)

caliblogits, caliblabels = split_logits_labels(model, calibloader, device)
testlogits, testlabels = split_logits_labels(model, testloader, device)

NameError: name 'build_dataset' is not defined

In [ ]:
caliprobs = F.softmax(caliblogits, dim=1)
calibpreds = torch.argmax(caliprobs, dim=1)
s_cal, _ = torch.max(caliprobs, dim=1)
incorrect_mask = calibpreds != caliblabels
s_cal = s_cal[incorrect_mask]

testprobs = F.softmax(testlogits, dim=1)
testpreds = torch.argmax(testprobs, dim=1)
s_test, _ = torch.max(testprobs, dim=1)